In [ ]:
from scipy.stats import zscore
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from scipy.fft import fft
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/project

/content/drive/MyDrive/project


In [ ]:
L=8064
Fs=128
Features=np.zeros((25600,32,3*128))
Labels=list()
for subject_id in range(0,32):
    if subject_id % 10 ==0:
      print(subject_id)
    loaded_data=None
    file_name=None
    if subject_id<9:
        file_name= 'data_preprocessed_python/s0'+str(subject_id+1)+'.dat'
    else:
        file_name= 'data_preprocessed_python/s'+str(subject_id+1)+'.dat'
    with open(file_name, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        loaded_data = u.load()
    data=loaded_data['data']
    Labels.append(loaded_data['labels'])
    
    for i in range(len(data)):
        for k in range(3,61,3):
          for j in range(32):   
            x=np.array(data[i][j][k*128:(k+3)*128]) -np.array((data[i][j][:3*128]))
            x=zscore(x)
            Features[subject_id*800+i*20+int(k/3-1)][j]=x

0
10
20
30


In [ ]:

def EEGNet(nb_classes, Chans = 64, Samples = 128, 
             dropoutRate = 0.5, kernLength = 64, F1 = 8, 
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense', 
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)



In [ ]:
y=np.zeros(800*32)
for j in range(32):
  for i in range(40):
    if Labels[j][i][1]>5:
      y[j*800+i*20:j*800+(i+1)*20]=1

In [ ]:
def run(train_index,test_index,
        dropout_rate=0.5,F1 = 8,D = 2, F2 = 16):



  Y=np_utils.to_categorical(y)

  d_size, chans, samples = Features.shape
  


  model = EEGNet(nb_classes = 2, Chans = chans, Samples = samples, 
               dropoutRate = dropout_rate, kernLength = 32, F1 = 8, D = 2, F2 = 16, 
               dropoutType = 'Dropout')
 

  # compile the model and set the optimizers
  model.compile(loss='categorical_crossentropy', optimizer='adam', 
                metrics = ['accuracy'])
  
  # count number of parameters in the model
  checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                                save_best_only=True)
  
  
  class_weights = {0:np.sum(y)/(len(y)-np.sum(y)), 1:1}
  
  fittedModel = model.fit(Features[train_index], Y[train_index], batch_size = 64, epochs =30 , 
                          verbose=2, validation_data=(Features[test_index], Y[test_index]),
                          callbacks=[checkpointer],class_weight=class_weights)
  probs       = model.predict(Features[test_index])
  preds       = probs.argmax(axis = -1)
  test = Y[test_index].argmax(axis=-1)
  acc = accuracy_score(test, preds)
  cm = confusion_matrix(test, preds)
  f1= f1_score(test, preds)
  return test, preds, acc,f1 , cm, model
  

In [ ]:
kf=KFold(n_splits=5,shuffle=True,random_state=0)

In [ ]:
train_index=list(kf.split(Features))[0][0]
test_index=list(kf.split(Features))[0][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 176s - loss: 0.7601 - accuracy: 0.5976 - val_loss: 0.6378 - val_accuracy: 0.6402

Epoch 00001: val_loss improved from inf to 0.63785, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 174s - loss: 0.7338 - accuracy: 0.6331 - val_loss: 0.6258 - val_accuracy: 0.6428

Epoch 00002: val_loss improved from 0.63785 to 0.62580, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 173s - loss: 0.7234 - accuracy: 0.6452 - val_loss: 0.6165 - val_accuracy: 0.6654

Epoch 00003: val_loss improved from 0.62580 to 0.61651, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 172s - loss: 0.7161 - accuracy: 0.6529 - val_loss: 0.6128 - val_accuracy: 0.6684

Epoch 00004: val_loss improved from 0.61651 to 0.61276, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 173s - loss: 0.7076 - accuracy: 0.6570 - val_loss: 0.6137 - val_accuracy: 0.6656

Epoch 00005: val_loss did not improve from 0.61276
Epoch 6/30
320/320 - 174s - loss: 0.7022 - accuracy: 0.6654 - val_loss: 0.

In [ ]:
resuslt

(array([1, 1, 1, ..., 0, 0, 0]),
 array([1, 0, 0, ..., 1, 1, 1]),
 0.7357421875,
 0.7618377046294666,
 array([[1603,  540],
        [ 813, 2164]]),
 <keras.engine.functional.Functional at 0x7fe3eaabf650>)

In [ ]:
train_index=list(kf.split(Features))[1][0]
test_index=list(kf.split(Features))[1][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 175s - loss: 0.7637 - accuracy: 0.6027 - val_loss: 0.6336 - val_accuracy: 0.6512

Epoch 00001: val_loss improved from inf to 0.63360, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 174s - loss: 0.7369 - accuracy: 0.6307 - val_loss: 0.6116 - val_accuracy: 0.6666

Epoch 00002: val_loss improved from 0.63360 to 0.61165, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 176s - loss: 0.7256 - accuracy: 0.6448 - val_loss: 0.6116 - val_accuracy: 0.6637

Epoch 00003: val_loss improved from 0.61165 to 0.61159, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 175s - loss: 0.7179 - accuracy: 0.6476 - val_loss: 0.6072 - val_accuracy: 0.6717

Epoch 00004: val_loss improved from 0.61159 to 0.60724, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 175s - loss: 0.7136 - accuracy: 0.6562 - val_loss: 0.6049 - val_accuracy: 0.6742

Epoch 00005: val_loss improved from 0.60724 to 0.60491, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 174s - loss: 

In [ ]:
resuslt

(array([1, 1, 1, ..., 0, 0, 0]),
 array([1, 1, 0, ..., 1, 1, 1]),
 0.7384765625,
 0.7694956102599416,
 array([[1546,  654],
        [ 685, 2235]]),
 <keras.engine.functional.Functional at 0x7fe3eaa90e10>)

In [ ]:
train_index=list(kf.split(Features))[2][0]
test_index=list(kf.split(Features))[2][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 14s - loss: 0.7615 - accuracy: 0.5341 - val_loss: 0.6763 - val_accuracy: 0.5650

Epoch 00001: val_loss improved from inf to 0.67628, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 11s - loss: 0.7415 - accuracy: 0.5841 - val_loss: 0.6563 - val_accuracy: 0.5996

Epoch 00002: val_loss improved from 0.67628 to 0.65632, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 11s - loss: 0.7307 - accuracy: 0.6026 - val_loss: 0.6424 - val_accuracy: 0.6178

Epoch 00003: val_loss improved from 0.65632 to 0.64239, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 11s - loss: 0.7225 - accuracy: 0.6124 - val_loss: 0.6411 - val_accuracy: 0.6240

Epoch 00004: val_loss improved from 0.64239 to 0.64106, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 11s - loss: 0.7161 - accuracy: 0.6198 - val_loss: 0.6376 - val_accuracy: 0.6498

Epoch 00005: val_loss improved from 0.64106 to 0.63758, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 11s - loss: 0.7104

In [ ]:
resuslt

(array([1, 1, 1, ..., 0, 1, 1]),
 array([1, 1, 1, ..., 1, 0, 0]),
 0.7181640625,
 0.7358594179022516,
 array([[1667,  608],
        [ 835, 2010]]),
 <keras.engine.functional.Functional at 0x7ff64594b610>)

In [ ]:
train_index=list(kf.split(Features))[3][0]
test_index=list(kf.split(Features))[3][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 41s - loss: 0.7561 - accuracy: 0.5547 - val_loss: 0.6819 - val_accuracy: 0.5561

Epoch 00001: val_loss improved from inf to 0.68186, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 11s - loss: 0.7342 - accuracy: 0.5959 - val_loss: 0.6553 - val_accuracy: 0.6092

Epoch 00002: val_loss improved from 0.68186 to 0.65535, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 11s - loss: 0.7244 - accuracy: 0.6111 - val_loss: 0.6468 - val_accuracy: 0.6326

Epoch 00003: val_loss improved from 0.65535 to 0.64682, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 11s - loss: 0.7153 - accuracy: 0.6277 - val_loss: 0.6344 - val_accuracy: 0.6398

Epoch 00004: val_loss improved from 0.64682 to 0.63442, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 11s - loss: 0.7067 - accuracy: 0.6347 - val_loss: 0.6287 - val_accuracy: 0.6578

Epoch 00005: val_loss improved from 0.63442 to 0.62873, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 12s - loss: 0.6981

In [ ]:
resuslt

(array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 0, 0, ..., 0, 0, 0]),
 0.7140625,
 0.7122641509433962,
 array([[1844,  454],
        [1010, 1812]]),
 <keras.engine.functional.Functional at 0x7f2580337890>)

In [ ]:
train_index=list(kf.split(Features))[4][0]
test_index=list(kf.split(Features))[4][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 13s - loss: 0.7598 - accuracy: 0.5455 - val_loss: 0.6753 - val_accuracy: 0.5941

Epoch 00001: val_loss improved from inf to 0.67529, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 11s - loss: 0.7415 - accuracy: 0.5896 - val_loss: 0.6592 - val_accuracy: 0.6070

Epoch 00002: val_loss improved from 0.67529 to 0.65919, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 11s - loss: 0.7297 - accuracy: 0.6035 - val_loss: 0.6484 - val_accuracy: 0.6209

Epoch 00003: val_loss improved from 0.65919 to 0.64842, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 11s - loss: 0.7220 - accuracy: 0.6130 - val_loss: 0.6411 - val_accuracy: 0.6375

Epoch 00004: val_loss improved from 0.64842 to 0.64108, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 11s - loss: 0.7140 - accuracy: 0.6239 - val_loss: 0.6381 - val_accuracy: 0.6410

Epoch 00005: val_loss improved from 0.64108 to 0.63805, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 11s - loss: 0.7043

In [ ]:
resuslt

(array([1, 1, 1, ..., 1, 1, 1]),
 array([0, 0, 0, ..., 0, 1, 0]),
 0.7046875,
 0.7183308494783904,
 array([[1680,  664],
        [ 848, 1928]]),
 <keras.engine.functional.Functional at 0x7f258033d3d0>)